In [9]:
import numpy as np
import pandas as pd
import os
import csv

In [22]:
gdas = []
with open("./BeFree/n_out/cooc.befree") as f:
    for line in f:
        items = line.strip().split('\t')
        gdas.append([int(items[0].split('|')[0]),"F", items[9], items[9], int(items[7].split('|')[0]), items[10], items[11], items[13], items[16], items[17], items[19]])

In [23]:
gdas_befree = pd.DataFrame(np.array(gdas), columns=["GAD_ID","GAD_ASSOC","GAD_GENE_SYMBOL","GAD_GENE_NAME","GAD_ENTREZ_ID","NER_GENE_ENTITY","NER_GENE_OFFSET","GAD_DISEASE_NAME","NER_DISEASE_ENTITY","NER_DISEASE_OFFSET","GAD_CONCLUSION"])
gdas_befree.GAD_ID = gdas_befree.GAD_ID.astype(str)
gdas_befree.GAD_ENTREZ_ID = gdas_befree.GAD_ENTREZ_ID.astype(str)


gdas_for_biobert = gdas_befree

In [24]:
# read loabel

In [25]:
gdas_for_biobert.shape

(3505, 11)

In [26]:
label_df = pd.read_csv('./BeFree/n_out/labels.txt', dtype={'pmid':str, 'geneId':str})

In [27]:
label_df

,pmid,geneId,diseaseId,label
0,1363780,1287,D009394,1.0
1,1678058,268,D000544,0.0
2,1678058,351,D000544,1.0
3,1901333,3458,C536141,0.5
4,1901333,3458,D007239,0.0
...,...,...,...,...
2729,25705178,3784,D029593,1.0
2730,25705178,3784,D029597,1.0
2731,26383258,4842,D010300,1.0
2732,26383258,4843,D010300,0.0


In [28]:
merge_df = gdas_for_biobert.merge(label_df, how='left', left_on=['GAD_ID', 'GAD_ENTREZ_ID', 'GAD_DISEASE_NAME'], \
                       right_on = ['pmid', 'geneId', 'diseaseId'])
merge_df['label'] = merge_df['label'].fillna(0)

In [29]:
gdas_for_biobert.head()

,GAD_ID,GAD_ASSOC,GAD_GENE_SYMBOL,GAD_GENE_NAME,GAD_ENTREZ_ID,NER_GENE_ENTITY,NER_GENE_OFFSET,GAD_DISEASE_NAME,NER_DISEASE_ENTITY,NER_DISEASE_OFFSET,GAD_CONCLUSION
0,25410422,F,twist2,twist2,117581,TWIST2,74#80,C536385,Setleis syndrome,0#16,"Setleis syndrome: clinical, molecular and stru..."
1,16076618,F,at1,at1,185,AT1,68#71,D011225,preeclampsia,152#164,CONCLUSIONS: Observed overrepresentation of mu...
2,16126967,F,hla,hla,3123,HLA,47#50,D001172,RA,140#142,"Our data suggest that in northeastern Hungary,..."
3,16126967,F,dr4,dr4,3126,DR4,51#54,D001172,RA,140#142,"Our data suggest that in northeastern Hungary,..."
4,16126967,F,drb1,drb1,3123,DRB1,79#83,D001172,RA,140#142,"Our data suggest that in northeastern Hungary,..."


In [30]:
merge_df[merge_df['label'] == 1][['GAD_ID', 'GAD_ENTREZ_ID', 'GAD_DISEASE_NAME']].drop_duplicates()

,GAD_ID,GAD_ENTREZ_ID,GAD_DISEASE_NAME
0,25410422,117581,C536385
1,16076618,185,D011225
2,16126967,3123,D001172
13,18093316,2944,D000236
15,18093316,2944,D015179
...,...,...,...
3491,20191019,4524,D020520
3496,18762705,6331,D053840
3498,7937790,3561,D053632
3500,17614770,6319,D049970


In [31]:
# gdas_for_biobert.iloc[4].GAD_CONCLUSION

In [32]:
print(gdas_for_biobert[['NER_GENE_OFFSET', 'NER_DISEASE_OFFSET', 'GAD_CONCLUSION']].head(3))

  NER_GENE_OFFSET NER_DISEASE_OFFSET  \
0           74#80               0#16   
1           68#71            152#164   
2           47#50            140#142   

                                      GAD_CONCLUSION  
0  Setleis syndrome: clinical, molecular and stru...  
1  CONCLUSIONS: Observed overrepresentation of mu...  
2  Our data suggest that in northeastern Hungary,...  


In [33]:
merge_df.shape

(3505, 15)

In [34]:
bert_df = merge_df[['label']].copy()

In [35]:
def get_snt(row):
    g_o = [int(i) for i in row['NER_GENE_OFFSET'].split('#')]
    d_o = [int(i) for i in row['NER_DISEASE_OFFSET'].split('#')]
    new_g, new_d = '@GENE$', '@DISEASE$'
    snt = row['GAD_CONCLUSION']
    _snt = ''
    # replace string
    if g_o[0] < d_o[0]:
        _snt = snt[:g_o[0]] + new_g
        _snt += snt[g_o[1]:d_o[0]] + new_d
        _snt += snt[d_o[1]:]
    else:
        _snt = snt[:d_o[0]] + new_d
        _snt += snt[d_o[1]:g_o[0]] + new_g
        _snt += snt[g_o[1]:]
    return _snt
bert_df['sentence'] = merge_df.apply(get_snt, axis = 1)

In [36]:
# bert_df.iloc[4].sentence

In [37]:
bert_df[bert_df['label'] == 0.5] = 0

In [38]:
bert_df['index'] = bert_df.index

In [39]:
bert_df = bert_df[['index', 'sentence', 'label']]
bert_df = bert_df.astype({"label": int})

In [40]:
bert_df.head()

,index,sentence,label
0,0,"@DISEASE$: clinical, molecular and structural ...",1
1,1,CONCLUSIONS: Observed overrepresentation of mu...,1
2,2,"Our data suggest that in northeastern Hungary,...",1
3,3,"Our data suggest that in northeastern Hungary,...",0
4,4,"Our data suggest that in northeastern Hungary,...",1


In [42]:
bert_df.to_csv('test.tsv', sep='\t', index=False)